### Composition

Wir lösen die Problematik durch Komposition (Composition) bzw. dem sogenannten Strategy-Pattern.

In [ ]:
class Shipment:
    def __init__(self,
                 carrier,
                 weight=0,
                 is_oversized=False,
                 is_hazardous=False,
                 surcharges=None):
        self.carrier = carrier
        self.weight = weight
        self.is_oversized = is_oversized
        self.is_hazardous = is_hazardous
        if surcharges is None:
            self.surcharges = []
        else:
            self.surcharges = surcharges

    def calculate_cost(self):
        cost = self.carrier.calculate_cost(self)
        for surcharge in self.surcharges:
            cost = cost + surcharge.apply(self)

        return cost


# Carrier Strategien
class Carrier:
    def calculate_cost(self, shipment):
        pass


class CarrierDHL(Carrier):
    def calculate_cost(self, shipment):
        if shipment.weight < 100:
            return 5
        else:
            return 10


class CarrierDPD(Carrier):
    def calculate_cost(self, shipment):
        if shipment.weight < 50:
            return 4
        else:
            return 8


# Zuschläge (Surcharges Strategien)
class Surcharges:
    def apply(self, shipment):
        pass


class SurchargesOversized(Surcharges):
    def apply(self, shipment):
        if shipment.is_oversized == True:
            return 10
        else:
            return 0


class SurchargesHazardous(Surcharges):
    def apply(self, shipment):
        if shipment.is_hazardous == True:
            return 100
        else:
            return 0


In [ ]:
shipment_oversized_hazardous = Shipment(
    carrier=CarrierDPD(),
    weight=200,
    is_oversized=True,
    is_hazardous=True,
    surcharges=[SurchargesOversized(), SurchargesHazardous()]
)

print(shipment_oversized_hazardous.calculate_cost())

In [ ]:
shipment_oversized = Shipment(
    carrier=CarrierDPD(),
    weight=200,
    is_oversized=True,
    is_hazardous=False,
    surcharges=[SurchargesOversized(), SurchargesHazardous()]
)

print(shipment_oversized.calculate_cost())

Wir wollen den Code nun so erweitern, dass Lieferungen ohne Sperrgut 500 kg nicht überschreiten.
Hierzu erweitern wir die `Shipment`-Klasse um ein Attribut `validators` (Validierungsstrategien) und führen eine neue Klasse für diese Validierungsstrategie ein.

In [ ]:
class Shipment:
    def __init__(self,
                 carrier,
                 weight=0,
                 is_oversized=False,
                 is_hazardous=False,
                 surcharges=None,
                 validators=None):  ## NEU
        self.weight = weight
        self.is_oversized = is_oversized
        self.is_hazardous = is_hazardous
        self.carrier = carrier
        if surcharges is None:
            self.surcharges = []
        else:
            self.surcharges = surcharges
        if validators is None:  ## NEU
            self.validators = []
        else:
            self.validators = validators

    def calculate_cost(self):
        try:  ## NEU
            for validator in self.validators:
                validator.validate(self)
        except ValueError as error:
            print(error)
            return None

        cost = self.carrier.calculate_cost(self)
        for surcharge in self.surcharges:
            cost = cost + surcharge.apply(self)

        return cost


class Validator:
    def validate(self, shipment):
        pass


class ValidatorMaxWeight(Validator):
    def validate(self, shipment):
        if shipment.is_oversized == False and shipment.weight > 500:
            raise ValueError("Maximales Gewicht überschritten")

In [ ]:
shipment_600kg = Shipment(
    carrier=CarrierDPD(),
    weight=600,
    is_oversized=False,
    validators=[ValidatorMaxWeight()]
)
print(shipment_600kg.calculate_cost())

In [ ]:
shipment_oversized_700kg = Shipment(
    carrier=CarrierDPD(),
    weight=700,
    is_oversized=True,
    surcharges=[SurchargesOversized(), SurchargesHazardous()],
    validators=[ValidatorMaxWeight()]
)

print(shipment_oversized_700kg.calculate_cost())